In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch
import matplotlib.pyplot as plt


In [ ]:
data = pd.read_csv('dataset.csv',header=None,names=['A','B','label'])
data = data.sample(frac=1).reset_index(drop=True)
train_data, test_data = train_test_split(data, test_size=0.1)

X_train = train_data[['A','B']].values;
Y_train = train_data['label'].values;
X_test = test_data[['A','B']].values;
Y_test = test_data['label'].values;


In [ ]:
plt.figure(figsize=(8, 6))

for label in train_data['label'].unique():
    subset = train_data[train_data['label'] == label]
    plt.scatter(
        subset['A'],
        subset['B'],
        label=f"Label {label}",
    )
plt.xlabel("Feature A", fontsize=12)
plt.ylabel("Feature B", fontsize=12)
plt.legend(title="Labels")
plt.grid(alpha=0.5)
plt.show()

In [ ]:

class FeedForwardNN(nn.Module):
    def __init__(self, N_fc1, N_fc2):
        super(FeedForwardNN, self).__init__()
        self.fc1 = nn.Linear(2, N_fc1)
        self.fc2 = nn.Linear(N_fc1, N_fc2)
        self.fc3 = nn.Linear(N_fc2, 1)
        
        self.relu = nn.ReLU()
        # self.relu = nn.Softmax()
        # self.relu = nn.LeakyReLU()
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x
    
Neurons = [(16, 16), (32, 64), (64, 64), (64, 128), (128, 128)]

N_fc1, N_fc2 = Neurons[0]



In [ ]:
model = FeedForwardNN(N_fc1, N_fc2)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# 定义训练循环
epochs = 100  # 训练的轮次
for epoch in range(epochs):
    model.train()  # 模型切换到训练模式
    optimizer.zero_grad()  # 清空梯度

    # 前向传播
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)  # 计算损失

# 反向传播和优化
    loss.backward()  # 反向传播
    optimizer.step()  # 更新权重

# 打印每 10 个 epoch 的损失
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")


In [ ]:
# 测试模型
model.eval()  # 切换到评估模式
with torch.no_grad():  # 禁用梯度计算
    y_pred = model(X_test)
    y_pred_classes = (y_pred >= 0.5).float()  # 将输出转换为 0 或 1

# 计算准确率
accuracy = torch.accuracy_score(Y_test, y_pred_classes.numpy())
print(f"Test Accuracy: {accuracy:.4f}")